In [4]:
import scipy as sp
import numpy as np
import math
import pandas as pd
from mpmath import besseli
import utm
import pyproj, math, datetime
from statistics import mode

In [17]:
samplingInterval = 60
timeWindow = 51
cutlength = 45
cutv = 4.1667
constv = 34.7/3.6

def Likelihoodww(data1,data2,cv): # calculate log-likelihood of the model
    def f(par):
        a = par[1]
        b = cv/sp.gamma(1+1/a)
        mx = par[2]
        my = par[3]
        wx = par[4]
        wy = par[5]
        L = 0
        for i in range(len(data1)):
            rr = np.sqrt((data1[i]*np.cos(data2[i]) - wx)**2 + (data1[i]*np.sin(data2[i]) - wy)**2)
            rx = (data1[i]*np.cos(data2[i])-wx)/rr
            ry = (data1[i]*np.sin(data2[i])-wy)/rr
            lp = (a-2)*math.log(rr) - (rr/b)**a + mx*rx + my*ry + math.log(a) - math.log(b) + (1-a)*math.log(b) - math.log(besseli(np.sqrt(mx**2 + my**2),0,))
            L = L+lp
        return L

def Weibull_sd(a,b): # standard deviation of Weibull distribution
    return b*np.sqrt(sp.gamma(1+2/a) - sp.gamma(1+1/a)*sp.gamma(1+1/a))

def Weibull_mean(a,b): # mean of Weibull distribution
    return b*sp.gamma(1+1/a)

def Von_Mises_sd(kappa): # standard deviation of von Mises distribution
    return 1/np.sqrt(kappa)

def readAxyGPS(filename): # read in AxyTrek GPS data (txt files)
    df = pd.read_csv(filename, sep = "\t", header = None, usecols = [0,1,2,3],
    names = ['Date','Time','lat','lon'])
    df['DT'] = pd.to_datetime(df['Date'] + " " + df['Time'],format="%d/%m/%Y %H:%M:%S")
    return df

def nearest(items, pivot): # find the nearest time position
    return min(items, key=lambda x: abs(x - pivot))

def timeRescale(dat,tdiff): # calculated indeces for rescaling time (DT) for regular sampling every tdiff mins
    return dat.iloc[np.arange(0,len(dat),step=np.timedelta64(tdiff,'m')/np.timedelta64(mode(np.diff(dat['DT'])),'s')).astype(int),:]

def spTrav(DT,lat,lon,threshold=0): # speed from time (DT), lat, and lon
    geod = pyproj.Geod(ellps='WGS84')
    _, _, distance = geod.inv(lon[0:-2],lat[0:-2],lon[1:-1],lat[1:-1])
    speed = (distance*10**-3)/np.array(np.diff(DT)/np.timedelta64(3600,'s'))
    if threshold != 0:
        while np.nanmax(speed) > threshold:
            lat = lat[speed < threshold]
            lon = lon[speed < threshold]
            _, _, distance = geod.inv(lon[0:-2],lat[0:-2],lon[1:-1],lat[1:-1])
            speed = (distance*10**-3)/np.array(np.diff(DT)/np.timedelta64(3600,'s'))
    return distance, speed

def XYfromUTM(lat,lon):
    return utm.from_latlon(lat,lon)

In [22]:
from sys import platform
import os, re, glob, pyproj, math, datetime
if platform == "darwin":
    fileloc = "/Volumes/GoogleDrive-112399531131798335686/My Drive/PhD/Data/2018Shearwater/AxyTrek/"
else:
    fileloc = "I:/My Drive/PD/Data/2018Shearwater/AxyTrek/"
# list all files
files = glob.glob(fileloc + "**/*.txt")
tags = np.unique([re.search('([0-9\-]+)[\\\\|/]',f).group(1) for f in files])
dat = readAxyGPS(files[1]) # read in
minDat = timeRescale(dat,1) # convert to 1 min fs
dt = (np.diff(minDat['DT']) / np.timedelta64(1,'s')).astype(int)

ValueError: Could not convert object to NumPy timedelta

array([ 52,  61,  61, ...,  60,  61, 430])